In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import SimulationData as sd
import RFTrackTools as rfttools
import copy
import json

In [ ]:
from importlib import reload
reload(bd)
reload(rfttools)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# RF-Track, Volume vs. Lattice for the Positron Capture Linac V0 (Yongke Tool V3)

* Compare RF-Track simulations of the same physics with different modeling:
  * Tracking with Volume in the first part of the AMD + tracking with Lattice in a homogeneous solenoidal channel with RF acceleration (Yongke's variant)
  * Full tracking in Volume (Mattia's variant)
* Compare simulations with homogeneous solenoidal channel vs. realistic fieldmaps of solenoids

## Distributions at the end of the AMD

Location of the switch between Volume and Lattice in Yongke's model.

### Volume, Yongke's Variant (Original, Octave)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrackResults/'
OCTAVE_MATRIX_NAME = 'A_AMD'
RFTRACK_FORMAT = 'rftrack_xp_t'
FILTER_SPECS_SELECTOR = None
beamAmdOutYon, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=DISTR_REL_PATH, octaveMatrixName=OCTAVE_MATRIX_NAME, rftrackDfFormat=RFTRACK_FORMAT,
    filterSpecsSelector=FILTER_SPECS_SELECTOR, z=np.nan, pdgId=-11
)
beamAmdOutYon.describe()

### Volume, Mattia's Variant (Python)

#### AMD 3D Fieldmap

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrackResults/CaptureLinac/'
beamAmd3dOutTia = bd.load_standard_fwf(DISTR_REL_PATH)
beamAmd3dOutTia.describe()

#### AMD 1D Fieldmap

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrackResults/CaptureLinac/'
beamAmd1dOutTia = bd.load_standard_fwf(DISTR_REL_PATH)
beamAmd1dOutTia.describe()

### Compare Models

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
plotDefs = bd.set_plot_defs_from_distrs([beamAmdOutYon, beamAmd3dOutTia, beamAmd1dOutTia], setNames=plotSets)
_ = bd.plot_distr(
    [beamAmdOutYon, beamAmd3dOutTia, beamAmd1dOutTia],
    plotDefs, markerStyle=['o', 'x', '+'],
    legendLabels=[
        'Volume, AMD 3D',
        'Volume, AMD 3D',
        'Volume, AMD 1D'
    ]
)

# Distributions at the End of Capture Linac (After 5 RF Structures, ~ 200 MeV)

### Volume + Lattice, Yongke's Variant (Octave, Original)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrackResults/YongkeTool_V3/CaptureLinacUpTo200MeV_LBandLargeR/RF200MeVOutputPositrons_E6GeV_SpotSize0.5mm_EmittXY15um_ConvTarget5X0_HTS_0.5T_LargeRLband.dat'
# DISTR_REL_PATH = '../../RFTrack/YongkeTool_V3/Dat/LatestSim/RF200MeVOutputPositrons_E6GeV_SpotSize0.5mm_EmittXY15um_ConvTarget5X0_HTS_0.5T_LargeRLband.dat'
OCTAVE_MATRIX_NAME = 'A_RF'
RFTRACK_FORMAT = 'rftrack_xp_t'
FILTER_SPECS_SELECTOR = None
beamRfOutYon, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=DISTR_REL_PATH, octaveMatrixName=OCTAVE_MATRIX_NAME, rftrackDfFormat=RFTRACK_FORMAT,
    filterSpecsSelector=FILTER_SPECS_SELECTOR, z=np.nan, pdgId=-11
)
beamRfOutYon.describe()

### Volume, Mattia's Variant (Python)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_LBandLargeR/DistrOut_After1stTracking_6d.sdf_txt'
# DISTR_REL_PATH = '../../Results_CaptureLinac/LatestSim/DistrOut_After1stTracking_6d.sdf_txt'
beamRfOutTia = bd.load_standard_fwf(DISTR_REL_PATH)
beamRfOutTia.describe()

### Volume, Realistic Solenoids

In [ ]:
# DISTR_REL_PATH = '../../Data/RFTrackResults/CaptureLinac/'
DISTR_REL_PATH = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_AMD3D_rkf45_RFT0242p8_RFPhases171_RealSolenoids/DistrOut_After1stTracking_6d.sdf_txt'
beamRfOutRealSol = bd.load_standard_fwf(DISTR_REL_PATH)
beamRfOutRealSol.describe()

### Compare Distributions

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
plotDefs = bd.set_plot_defs_from_distrs([beamRfOutYon, beamRfOutTia], setNames=plotSets)
_ = bd.plot_distr(
    [beamRfOutYon, beamRfOutTia],
    plotDefs, markerStyle=['o', 'x'], legendLabels=['Volume + Lattice', 'Volume']
)

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamRfOutTia, beamRfOutRealSol], setNames=plotSets)
_ = bd.plot_distr(
    [beamRfOutTia, beamRfOutRealSol],
    plotDefs, markerStyle=['x', '+'], legendLabels=['Homog. sol. field', 'Realistic sol.']
)

## Transport Along the Linac

### Volume + Lattice (Yongke) vs. Volume Only (Mattia)

In [ ]:
reload(rfttools)

SIM_REL_PATH_VOL = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_LBandLargeR'
SIM_REL_PATH_VOLLAT = '../../Data/RFTrackResults/YongkeTool_V3/CaptureLinacUpTo200MeV_LBandLargeR'
#
# SIM_REL_PATH_VOL = '../../Results_CaptureLinac/LatestSim'
# SIM_REL_PATH_VOLLAT = '../../RFTrack/YongkeTool_V3/Dat/LatestSim'
#
fig1, ax1l = plt.subplots(3, 1, figsize=(9.6, 6.4))
ax1r = []
for ax in ax1l:
    ax1r.append(ax.twinx())
ax1 = [None] * (len(ax1l) + len(ax1r))
ax1[::2] = ax1l
ax1[1::2] = ax1r
sShiftEMFields = 236.3e-3  # [m]
rfttools.load_plot_transport(ax1, SIM_REL_PATH_VOLLAT, sShiftEMFields=sShiftEMFields)
rfttools.load_plot_transport(ax1, SIM_REL_PATH_VOL)
ax1[0].legend(['Volume + Lattice', 'Volume'], loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
_ = ax1[2].set_ylim([0, 200.])

In [ ]:
# SIM_REL_PATH_HOMOG = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_AMD3D_rkf45_RFT0304p7_RFPhases171'
# SIM_REL_PATH_REAL = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_AMD3D_rkf45_RFT0304p7_RFPhases171_RealSolenoids'
#
SIM_REL_PATH_HOMOG = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_AMD3D_rkf45_RFT0242p8_RFPhases171'
SIM_REL_PATH_REAL = '../../Data/RFTrackResults/CaptureLinac/CaptureLinacUpTo200MeV_AMD3D_rkf45_RFT0242p8_RFPhases171_RealSolenoids'
reload(rfttools)
fig1, ax1l = plt.subplots(3, 1, figsize=(9.6, 6.4))
ax1r = []
for ax in ax1l:
    ax1r.append(ax.twinx())
ax1 = [None] * (len(ax1l) + len(ax1r))
ax1[::2] = ax1l
ax1[1::2] = ax1r
sShiftEMFields = 0  #225.3e-3  # [m]
rfttools.load_plot_transport(ax1, SIM_REL_PATH_HOMOG, sShiftEMFields=sShiftEMFields)
rfttools.load_plot_transport(ax1, SIM_REL_PATH_REAL)
ax1[0].legend(['Homog. sol. field 0.5 T', 'Realistic solenoid'], loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
# ax1[0].set_ylim([0, 0.7])
# ax1[1].set_visible(False)
_ = ax1[2].set_ylim([0, 200.])

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>